# Notebook 1 – États EPR (Bell) & GHZ : théorie + circuits Qiskit

**Objectifs pédagogiques :**
- Comprendre la construction des états EPR (Bell) et GHZ.
- Savoir implémenter les circuits correspondants avec Qiskit (simulation locale).
- Observer les distributions de mesure et le vecteur d'état.

**Pré-requis :** Bases de Python, notions de qubit/superposition/intrication, matrices & Dirac.


## 0) Installation et imports

> Si Qiskit n'est pas déjà installé dans votre environnement, exécutez la cellule suivante.


In [ ]:
# Optionnel : décommentez si nécessaire
# !pip install qiskit qiskit-aer --quiet
# !pip install pylatexenc --quiet

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_vector
import numpy as np
import matplotlib.pyplot as plt


## 1) Rappel théorique – état EPR (Bell)

On vise l'état de Bell aussi appelé EPR.  
**Procédure standard (2 qubits)** :
1. Préparer l'état de départ.
2. Appliquer **Hadamard** sur le qubit 0.
3. Appliquer **CNOT** avec qubit **0** comme contrôle et qubit **1** comme cible.
4. On obtient le résultat désiré.

### 1.1) Circuit EPR (Hadamard + CNOT)

In [ ]:
# Construction de l'état EPR (|Φ+>)
qc_epr = QuantumCircuit(2, 2)
qc_epr.h(0)         # Hadamard sur qubit 0
qc_epr.cx(0, 1)     # CNOT: contrôle=0, cible=1
qc_epr.draw('mpl')


### 1.2) Vecteur d'état (Statevector)

On simule le vecteur d'état pour vérifier que l'on obtient bien l'état désiré.


In [ ]:
# Statevector (simulation exacte)
sv_epr = Statevector.from_instruction(qc_epr)
sv_epr


### 1.3) Mesure en base Z

On mesure plusieurs fois et on observe que seules les issues `00` et `11` apparaissent (≈ 50/50).


In [ ]:
# Ajout de la mesure et exécution sur Aer qasm_simulator
qc_epr_meas = qc_epr.copy()
qc_epr_meas.measure([0,1], [0,1])

sim = AerSimulator()
result = sim.run(qc_epr_meas, shots=4096).result()
counts = result.get_counts()# Affichage de l'histogramme
plot_histogram(counts)


## 2) Rappel théorique – état GHZ

L'état **GHZ à 3 qubits** le plus courant est :
\\[
\lvert \mathrm{GHZ}_3 \rangle = \tfrac{1}{\sqrt{2}}(\lvert 000 \rangle + \lvert 111 \rangle).
\\]

**Procédure standard (3 qubits)** :
1. Préparer \\(\lvert 0 0 0 \rangle\\).
2. Appliquer **Hadamard** sur le qubit 0.
3. Appliquer **CNOT** de 0 vers 1, puis **CNOT** de 0 vers 2.
4. On obtient \\(\lvert \mathrm{GHZ}_3 \rangle\\).

### 2.1) Circuit GHZ (3 qubits)

In [ ]:
# Construction de GHZ_3
qc_ghz = QuantumCircuit(3, 3)
qc_ghz.h(0)       # Superposition sur qubit 0
qc_ghz.cx(0, 1)   # CNOT 0->1
qc_ghz.cx(0, 2)   # CNOT 0->2
qc_ghz.draw('mpl')


### 2.2) Vecteur d'état GHZ

In [ ]:
sv_ghz = Statevector.from_instruction(qc_ghz)
sv_ghz


### 2.3) Mesure en base Z

On s'attend à voir `000` et `111` avec des fréquences ≈ 50/50.


In [ ]:
qc_ghz_meas = qc_ghz.copy()
qc_ghz_meas.measure([0,1,2], [0,1,2])

backend = AerSimulator()
result = AerSimulator().run(qc_ghz_meas, shots=4096).result()
counts_ghz = result.get_counts()
plot_histogram(counts_ghz)


## C'EST A VOUS DE JOUER !

## 3) Généralisation - GHZ à **N** qubits

- Commencez par écrire l'état GHZ à 4 puis à 5 qubits.
- Déduisez-en une idée de généralisation de GHZ à N qubits.


In [ ]:
# Construction de GHZ_4
qc_ghz4 = QuantumCircuit(4, 4)
qc_ghz4.h(0)
qc_ghz4.cx(0, 1)
qc_ghz4.cx(0, 2)
qc_ghz4.cx(0, 3)
qc_ghz4.draw('mpl')

In [ ]:
# Affichage de l'histogramme
qc_ghz_meas = qc_ghz4.copy()
qc_ghz_meas.measure([0,1,2,3], [0,1,2,3])

backend = AerSimulator()
result = AerSimulator().run(qc_ghz_meas, shots=4096).result()
counts_ghz = result.get_counts()
plot_histogram(counts_ghz)

In [ ]:
# Affichage de l'état
sv_ghz = Statevector.from_instruction(qc_ghz4)
sv_ghz

In [ ]:
# Construction de GHZ_5
qc_ghz = QuantumCircuit(5, 5)
qc_ghz5.h(0)
qc_ghz5.cx(0, 1)
qc_ghz5.cx(0, 2)
qc_ghz5.cx(0, 3)
qc_ghz5.cx(0, 4)
qc_ghz5.draw('mpl')

> **Généralisation à N qubits** : Hadamard sur le premier qubit, puis enchaîner des CNOT du premier qubit vers tous les autres (1→2, 1→3, …, 1→N-1).

## Exercice guidé – Construire une fonction GHZ généralisée

L’objectif est d’écrire une fonction `ghz_circuit(n_qubits, measure=True)` qui génère l’état GHZ à *n* qubits.

---

### Étape 1 : Créer un circuit vide
Paramètres : 
- le nombre de qubits : `n_qubits`  
- un booléen pour effectuer ou non une mesure, ajouter alors des bits classiques : `measure=False`.  

*Indice : utiliser `QuantumCircuit(...)`.*

---

### Étape 2 : Préparer la superposition
- Appliquer une porte **Hadamard** au premier qubit (qubit 0).  
- Cela crée l’état \((|0\rangle + |1\rangle)/\sqrt{2}\).  

*Indice : méthode `.h(...)`.*

---

### Étape 3 : Créer l’intrication avec les autres qubits
- Parcourir les qubits de 1 à `n_qubits-1`.  
- Appliquer une porte **CNOT** avec le qubit 0 comme contrôle, et chaque autre comme cible.  

*Indice : méthode `.cx(controle, cible)`.*

---

### Étape 4 : Ajouter les mesures (si demandé)
- Vérifier la valeur du paramètre `measure`.  
- Si `True`, ajouter des mesures de chaque qubit dans le registre classique correspondant.  

*Indice : méthode `.measure(range(...), range(...))`.*

---

### Étape 5 : Retourner le circuit
- La fonction doit renvoyer le `QuantumCircuit` final.  

---

### Facultatif : Ajouter une barrière
- Ajouter une barrière verticale sur tous les qubits pour améliorer la visibilité, juste avant d'effectuer la mesure.

*Indice : méthode `.barrier()`.*

👉 **But de l’exercice** : ne recopiez pas du code tout fait, mais suivez ce plan pas à pas pour écrire la fonction vous-mêmes.


In [ ]:
def ghz_circuit(n_qubits, measure=False):
    """Construit un circuit GHZ_n (|0...0> + |1...1>)/sqrt(2).
    Args:
        n_qubits: nombre de qubits (>= 2)
        measure: ajoute la mesure si True
    Returns:
        QuantumCircuit prêt à l'emploi.
    """
    if n_qubits < 2:
        raise ValueError("n_qubits doit être >= 2")

    qc = QuantumCircuit(n_qubits, n_qubits if measure else 0)
    qc.h(0)
    for k in range(1, n_qubits):
        qc.cx(0, k)
    qc.barrier()
    if measure:
        qc.measure(list(range(n_qubits)), list(range(n_qubits)))
    return qc

# Démo rapide
qc_ghz5 = ghz_circuit(5, measure=True)
qc_ghz5.draw('mpl')


## Exercices guidés

### Exercice 1 : Expérimenter avec une porte Z
Créez un circuit avec l'état de Bell.
Ajoutez une **porte Z** sur le premier qubit **avant la mesure**.  

**Question :** Qu’est-ce qui change dans la distribution des résultats par rapport à l’état de Bell initial ?  

---

### Exercice 2 : Superposition simple
Construisez un circuit avec **1 qubit** qui crée la superposition \\((|0\rangle + |1\rangle)/\sqrt{2}\\).  

**Question :** Qu'observez-vous et avec quelles probabilités ?  

---

### Exercice 3 : CNOT en action
Construisez un circuit de **2 qubits** avec l’état initial \\(|10\rangle\\).  

**Indications :**
1. Préparez le deuxième qubit dans l’état |1⟩.  
2. Appliquez une porte **CNOT** avec le qubit 1 comme contrôle et le qubit 0 comme cible.  
3. Mesure les deux qubits.  

**Question :** Quelle corrélation observez-vous entre les qubits ?  

In [ ]:
# Exercice 1
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1) 
qc.z(0)  # Ajout de Z sur le premier qubit (qubit 0) AVANT la mesure
qc.measure_all()
sim = AerSimulator()
counts = sim.run(qc, shots=4096).result().get_counts()
plot_histogram(counts)

# Exercice 2
qc = QuantumCircuit(1)
qc.h(0)          # superposition
qc.measure_all()
sim = AerSimulator()
counts = sim.run(qc, shots=4096).result().get_counts()
plot_histogram(counts)

# Exercice 3
qc = QuantumCircuit(2)
qc.x(1)          # met le 2e qubit (index 1) à |1> → état |01>
qc.cx(1, 0)      # contrôle = qubit 1, cible = qubit 0
qc.measure_all()
sim = AerSimulator()
counts = sim.run(qc, shots=4096).result().get_counts()
plot_histogram(counts)

---

**Crédits et notes :**
- Ce notebook s'exécute en **simulation locale** (Aer). Pour des exécutions sur matériel réel, configurez un compte IBM Quantum.
- Compatibilité : **Qiskit 1.x+**.

**Licence pédagogique** : libre d'usage dans le cadre de la masterclass (avec attribution).

Copyright © 2025 | Dr AMOUZOU Grâce Dorcas Akpéné